In [ ]:
import sys
import os
from pathlib import Path
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2
from rich.progress import Progress

from utils import get_abs_path
from a_star import find_path

In [ ]:
def cv2_imshow(img):
    img = np.concatenate((img[:,:,2:3], img[:,:,1:2], img[:,:,0:1]), axis=2)
    plt.imshow(img)
    plt.show()


def load_images(images_path, json_path):
    data = {}
    images = []
    files = []

    for (dirpath, dirnames, filenames) in os.walk(json_path):
        files.extend(filenames)
        break

    for (dirpath, dirnames, filenames) in os.walk(images_path):
        images.extend(filenames)
        break

    images = [f for f in images if f.endswith('.png')]

    # remove already generated paths
    for file in files:
        # remove '.json' and add '.png'
        file = file[0:-5] + '.png'
        if file in images:
            images.remove(file)

    images = sorted(images)
    with Progress() as progress:
        task = progress.add_task('[yellow]Loading images...', total=len(images))

        for i in range(len(images)):
            img_path = images_path + images[i]
            img = cv2.imread(img_path)
            data[images[i]] = img
            progress.update(task, advance=1)

    return data

In [ ]:
def get_coordinates(images_path, points_filename):
    coordinates = {}
    with open(images_path+points_filename, newline='') as f:
        reader = csv.reader(f)
        reader = list(reader)
        for line in reader:
            map_name = line[0]
            points_coordinates = line[1:]
            coordinates[map_name] = points_coordinates
    return coordinates


def load_data(images_path, json_path, points_filename):
    images = load_images(images_path, json_path)
    coordinates = get_coordinates(images_path, points_filename)
    return images, coordinates

## A* planning algorithm

In [ ]:
def mark_path_points(oryg_map, oryg_path):
    map = np.copy(oryg_map)
    path = oryg_path.copy()
    start_point = path[0]
    goal_point = path[-1]
    path = path[1:-1]

    # y, x
    directions_dict = {
        (-1,-1): 'NW',
        (-1, 0): 'N',
        (-1, 1): 'NE',
        ( 0, 1): 'E',
        ( 1, 1): 'SE',
        ( 1, 0): 'S',
        ( 1,-1): 'SW',
        ( 0,-1): 'W'
    }

    mark_points = []
    last_x = start_point[0]
    last_y = start_point[1]
    last_dir = None

    for curr_x, curr_y in path:
        x_change = curr_x - last_x
        y_change = curr_y - last_y
        dir = directions_dict[(y_change, x_change)]

        if dir is not last_dir:
            mark_points.append( (last_x, last_y) )

        last_x = curr_x
        last_y = curr_y
        last_dir = dir

    mark_points.append(goal_point)

    for x, y in mark_points:
        map[y, x, 1] = 0
        map[y, x, 2] = 255

    return map, mark_points

In [ ]:
def generate_point_paths(dataset, coordinates, results_path):

    size = len(dataset)
    with Progress() as progress:
        task = progress.add_task('[yellow]Loading images...', total=size)

    for map_name in dataset:
        paths = {}
        print(map_name)
        map = dataset[map_name]
        coor = coordinates[map_name]

        map, path = find_path(map, coor)
        map, points = mark_path_points(map, path)

        paths[map_name] = points

        # remove .png from name and add json
        map_name = map_name[0:-4] + '.json'
        # print(map_name)
        with open(results_path +  map_name, 'w') as f:
            json.dump(paths, f)

        # cv2_imshow(map)
        progress.update(task, advance=1)


In [ ]:
project_path = get_abs_path(1)
project_path += '/data/'
dirs = ['train', 'valid', 'test']
maps_points_filename = 'maps_points.csv'

for dir in dirs:
    print(dir)
    images_path = project_path + dir + '/maps/'
    json_path = project_path + dir + '/planned_maps/'
    Path(images_path).mkdir(parents=True, exist_ok=True)
    Path(json_path).mkdir(parents=True, exist_ok=True)

    dataset, coordinates = load_data(images_path, json_path, maps_points_filename)

    for key in dataset:
        cv2_imshow(dataset[key])
        break

    generate_point_paths(dataset, coordinates, json_path)